In [1]:
import pandas as pd
from datetime import datetime as dt

import integrate as intr



In [2]:

miv_df = intr.process_miv_data()
#fb_data = intr.process_foot_bike_data()

/Users/seb/Projects/repos/group-1/src/integrate.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  miv_df_cols_dropped['Weekday_en'] = days.map(lambda x: weekday_names[x])
/Users/seb/Projects/repos/group-1/src/integrate.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  miv_df_cols_dropped['AnzFahrzeuge'] = miv_df_cols_dropped['AnzFahrzeuge'].fillna(0).astype(int)


In [6]:
duplicate_rows = miv_df[miv_df.duplicated()]
print(duplicate_rows.shape[0])

121


In [ ]:
acc_df = intr.process_accident_data(True)

In [ ]:
acc_df.head()
acc_df['AccidentWeekDay_en'].unique()
#acc_df.dtypes



In [ ]:
print("Accident Columns:")
print(acc_df.dtypes)
print()
print("MIV Columns:")
print(miv_df.dtypes)
print()
print("FB Cols:")
print(fb_data.dtypes)

In [ ]:
acc_df['ID'] = acc_df.index +1
acc_df[('ID')]

In [ ]:
print("MIV unqiue:", miv_df['EKoord'])
print("Acc unique:", acc_df['RoadType'].unique)
print("FB unique: ", fb_data['DATE'])


In [ ]:
acc_df.head()

In [ ]:
from sqlalchemy import create_engine
from geoalchemy2 import Geometry, WKTElement
import geopandas as gpd
from shapely import wkt

db_url = f'postgresql://seb:@localhost:5432/test-db23'
engine =  create_engine(db_url)

#miv_df.to_sql('table_name', engine, if_exists='replace', index=False)
#fb_data.to_sql('footbike', engine, if_exists='replace', index=False)

geometry_column = 'geometry'


acc_df['geometry'] = acc_df['geometry'].apply(lambda geom: WKTElement(geom.wkt, srid=4326))

acc_df.to_sql('accidents', engine, if_exists='replace', index=False, dtype={'geometry': Geometry('POINT', srid=4326)})



In [ ]:
engine.dispose()